# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data_file = "../output-data/cities.csv"

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv(output_data_file)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,wenceslau braz,-23.87,-49.80,82.00,75,23,3.00,BR,2020-10-20
1,ushuaia,-54.80,-68.30,42.80,60,75,23.04,AR,2020-10-20
2,piacabucu,-10.41,-36.43,82.80,59,15,13.15,BR,2020-10-20
3,freeport,40.66,-73.58,64.99,93,90,4.70,US,2020-10-20
4,leningradskiy,69.38,178.42,31.28,91,82,9.28,RU,2020-10-20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = df[["Lat", "Lng"]].astype("float")

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
hotel_df = df.loc[(df["Max Temp"]<85) & 
                  (df["Max Temp"]>70) & 
                  (df["Cloudiness"]<20) & 
                  (df["Humidity"]<50) & 
                  (df["Wind Speed"]<10)]

hotel_df = hotel_df.dropna(how='any')
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,keshorai patan,25.30,75.93,80.40,25,0,6.73,IN,2020-10-20
1,riyadh,24.69,46.72,84.20,11,0,9.17,SA,2020-10-20
2,susangerd,31.56,48.18,75.20,31,0,2.24,IR,2020-10-20
3,chitungwiza,-18.01,31.08,75.78,22,0,6.55,ZW,2020-10-20
4,matamoros,25.53,-103.25,71.60,26,5,3.85,MX,2020-10-20
5,yulara,-25.24,130.99,73.40,49,0,9.17,AU,2020-10-20
6,waddan,29.16,16.14,78.13,34,2,8.14,LY,2020-10-20
7,tabas,33.60,56.92,75.20,9,0,6.93,IR,2020-10-20
8,buraydah,26.33,43.98,84.20,18,0,5.82,SA,2020-10-20
9,uthal,25.81,66.62,79.84,20,0,2.62,PK,2020-10-20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""

target_type = "lodging"
radius = 5000

In [7]:

for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    target_coordinates = f"{row['Lat']}, {row['Lng']}"

    params = {
            "location": target_coordinates,
            "radius": radius,
            "type": target_type,
            "key": g_key
        }
    try:
        # make request
        hotels = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = float('NaN')

hotel_df = hotel_df.dropna(how='any')
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,keshorai patan,25.30,75.93,80.40,25,0,6.73,IN,2020-10-20,Hotel Jamindar Palace
1,riyadh,24.69,46.72,84.20,11,0,9.17,SA,2020-10-20,InterContinental Riyadh
2,susangerd,31.56,48.18,75.20,31,0,2.24,IR,2020-10-20,فاطمه ابرفت
3,chitungwiza,-18.01,31.08,75.78,22,0,6.55,ZW,2020-10-20,Damview Hotel
4,matamoros,25.53,-103.25,71.60,26,5,3.85,MX,2020-10-20,Recreativo Sabdy Briseidy
5,yulara,-25.24,130.99,73.40,49,0,9.17,AU,2020-10-20,Desert Gardens Hotel - Ayers Rock Resort
6,waddan,29.16,16.14,78.13,34,2,8.14,LY,2020-10-20,محل الشريف محمد العايب
7,tabas,33.60,56.92,75.20,9,0,6.93,IR,2020-10-20,شرکت معماری مهراز شایگان آراد
8,buraydah,26.33,43.98,84.20,18,0,5.82,SA,2020-10-20,Mövenpick Hotel Qassim
9,uthal,25.81,66.62,79.84,20,0,2.62,PK,2020-10-20,Al Madina Hotal


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))